# 데이터 전처리
+ 데이터 분석 과정에서 반드시 거쳐야 하는 과정
+ 전처리를 어떻게 하느냐에 따라 결과에 크게 영향을 줌


## 데이터 인코딩
### 1. 레이블 인코딩
+ **카테고리형 피처를 코드형 숫자값으로 변환하는 것**
+ 예) 상품 데이터에 대해서 TV, 냉장고, 전자레인지, 컴퓨터 등으로 상품 구분이 되어있을 때, TV: 1, 냉장고: 2, 전자레인지: 3, 컴퓨터: 4 와 같은 숫자형으로 변환한다.
+ 사이킷런의 **LabelEncoder 클래스**

In [3]:
from sklearn.preprocessing import LabelEncoder

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# LabelEncdoer 객체 생성
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print('인코딩 변환값: ', labels)

인코딩 변환값:  [0 1 4 5 3 3 2 2]


TV: 0, 냉장고: 1, 전자레인지: 4, 컴퓨터: 5, 선풍기:3, 믹서: 2 로 변환됨

In [8]:
# 어떤 문자열 값이 어떤 숫자값으로 인코딩되었는지 확인
# classes_ 속성 사용
print('인코딩 클래스', encoder.classes_)

인코딩 클래스 ['TV' '냉장고' '믹서' '선풍기' '전자레인지' '컴퓨터']


In [10]:
# 인코딩된 값을 다시 디코딩
# inverse_transform() 사용
print('디코딩 원본값', encoder.inverse_transform([4, 5, 2, 0, 1, 1, 3, 3]))

디코딩 원본값 ['전자레인지' '컴퓨터' '믹서' 'TV' '냉장고' '냉장고' '선풍기' '선풍기']


+ 레이블 인코딩의 경우, 숫자 값으로 변환되기 때문에 ML 알고리즘에 따라 예측 성능이 떨어질 수도 있다.
    + 변환된 숫자값이 가중치가 더 부여된 것으로 인식될 가능성이 있다.
    + 선형회귀와 같은 ML 알고리즘에는 적용될 수 없다.

### 2. 원-핫 인코딩
+ 피처 값의 유형에 따라서 새로운 피처를 추가해 고유값에 해당하는 칼럼에만 1, 나머지 칼럼에는 9을 표시하는 것
+ 예) 원본 데이터에서 고유값이 ['TV', '냉장고', '전자레인지', '컴퓨터'] 일 때, 원-핫 인코딩하면 다음과 같다.

|상품분류|상품분류_TV|상품분류_냉장고|상품분류_전자레인지|상품분류_컴퓨터|
|:-:|:-:|:-:|:-:|:-:|
|TV| 1 | 0 | 0 | 0 | 
|냉장고| 0 | 1 | 0 | 0 |
|전자레인지| 0 | 0 | 1| 0 |
|컴퓨터| 0 | 0 | 0 | 1 |
|컴퓨터| 0 | 0 | 0 | 1 |

+ 사이킷런의 **OneHotEncoder 클래스**
+ OneHotEncoder 클래스를 사용할 때, 
    + 모든 문자열 값이 숫자형으로 변환되어야 한다.
    + 입력 값으로 2차원 데이터가 필요하다.

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np

items = ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']

# LabelEncoder를 사용하여 숫자값으로 변환
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
# 2차원 데이터로 변환
labels = labels.reshape(-1, 1)

# OneHotEncoder 객체 생성
oh_encoder = OneHotEncoder()
oh_encoder.fit(labels)
oh_labels = oh_encoder.transform(labels)

print('원-핫 인코딩 데이터')
print(oh_labels.toarray())
print('\n원-핫 인코딩 데이터 차원')
print(oh_labels.shape)

원-핫 인코딩 데이터
[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]]

원-핫 인코딩 데이터 차원
(8, 6)


#### get_dummies()
+ 판다스에서 지원하는 원-핫 인코딩 API
+ 숫자형 값으로 변환 없이도 바로 변환이 가능하다

In [15]:
import pandas as pd

df = pd.DataFrame({'item': ['TV', '냉장고', '전자레인지', '컴퓨터', '선풍기', '선풍기', '믹서', '믹서']})
pd.get_dummies(df)

,item_TV,item_냉장고,item_믹서,item_선풍기,item_전자레인지,item_컴퓨터
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,0,1,0,0
5,0,0,0,1,0,0
6,0,0,1,0,0,0
7,0,0,1,0,0,0


## 피처 스케일링과 정규화
+ **피처 스케일링** : 서로 다른 변수의 값 범위를 일정한 수준으로 맞추는 작업
    + 표준화(Standardization) : 데이터의 피처 각각이 평균=0, 분산=1인 가우시안 정규 분포를가진 값으로 변환하는 것
    + 정규화(Normalization) : 서로 다른 피처의 크기를 통일하기 위해 크기를 변환해주는 것
+ 사이킷런의 대표적인 피처 스케일링 : **StandardScaler, MinMaxScaler 클래스**

### StandardScaler
+ 표준화를 쉽게 지원하기 위한 클래스
+ **개별 피처를 평균=0, 분산=1 인 값으로 변환**
+ 서포트 벡터 머신, 선형 회귀, 로지스틱 회귀 등에서 표준화를 적용하는 것이 중요하다

In [16]:
from sklearn.datasets import load_iris
import pandas as pd

# 붓꽃 데이터 셋을 로딩하고 DataFrame으로 변환
iris = load_iris()
iris_data = iris.data
iris_df = pd.DataFrame(data=iris_data, columns=iris.feature_names)


# StandardScaler 전 평균, 분산 값
print('feature 들의 평균 값')
print(iris_df.mean())
print('\nfeature 들의 분산 값')
print(iris_df.var())

feature 들의 평균 값
sepal length (cm)    5.843333
sepal width (cm)     3.057333
petal length (cm)    3.758000
petal width (cm)     1.199333
dtype: float64

feature 들의 분산 값
sepal length (cm)    0.685694
sepal width (cm)     0.189979
petal length (cm)    3.116278
petal width (cm)     0.581006
dtype: float64


In [17]:
from sklearn.preprocessing import StandardScaler

# StandardScaler객체 생성
scaler = StandardScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform() 시에 nadarray 로 반환된 데이터 세트를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('feature 들의 평균 값')
print(iris_df_scaled.mean())
print('\nfeature 들의 분산 값')
print(iris_df_scaled.var())

feature 들의 평균 값
sepal length (cm)   -1.690315e-15
sepal width (cm)    -1.842970e-15
petal length (cm)   -1.698641e-15
petal width (cm)    -1.409243e-15
dtype: float64

feature 들의 분산 값
sepal length (cm)    1.006711
sepal width (cm)     1.006711
petal length (cm)    1.006711
petal width (cm)     1.006711
dtype: float64


모든 칼럼 값이 평균이 0에 가까운 값으로, 분산은 1에 가까운 값으로 변환됨

### MinMaxScaler
+ **데이터 값을 0과 1 사이의 범위 값으로 변환**
+ 데이터 분포가 가우시안 분포가 아닐 경우, MinMaxScaler 를 적용해볼 수 있다.

In [18]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
scaler.fit(iris_df)
iris_scaled = scaler.transform(iris_df)

# transform() 시에 nadarray 로 반환된 데이터 세트를 DataFrame으로 변환
iris_df_scaled = pd.DataFrame(data=iris_scaled, columns=iris.feature_names)

print('feature들의 최소 값')
print(iris_df_scaled.min())
print('\nfeature들의 최대 값')
print(iris_df_scaled.max())

feature들의 최소 값
sepal length (cm)    0.0
sepal width (cm)     0.0
petal length (cm)    0.0
petal width (cm)     0.0
dtype: float64

feature들의 최대 값
sepal length (cm)    1.0
sepal width (cm)     1.0
petal length (cm)    1.0
petal width (cm)     1.0
dtype: float64


모든 피처에 0~1 사이의 값으로 변환되는 스케일링이 적용됨

## 학습 데이터와 테스트 데이터의 스케일링 변환 시 유의점
+ Scaler 객체를 이용하여 데이터 스케일링 변환 시 **fit(), transform(), fit_transform()** 을 사용함

#### 학습 데이터 세트, 테스트 데이터 세트에 fit(), transform() 적용 시 주의점
--> 학습 데이터로 fit()이 적용된 스케일링 기준 정보를 그대로 테스트 데이터에 적용해야 한다.  
+ 테스트 데이터로 새로운 스케일링 기준 정보를 만들면 학습, 테스트 데이터의 스케일링 기준정보가 달라져서 올바른 예측 결과를 도출하기 어렵다

#### 테스트 데이터에 fit()적용 시 발생하는 문제 확인

In [19]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# 학습 데이터(train_array) - 0 부터 10까지 값을 가지는 데이터 세트로 생성
# 테스트 데이터(test_array) - 0 부터 5까지 값을 가지는 데이터 세트로 생성
# Scaler클래스에 적용하기 위해서 reshape(-1, 1)로 2차원으로 변경
train_array = np.arange(0, 11).reshape(-1, 1)
test_array =  np.arange(0, 6).reshape(-1, 1)

In [20]:
# MinMaxScaler객체 생성
scaler = MinMaxScaler()

# fit() 적용
# train_array 데이터의 최소값이 0, 최대값이 10으로 설정.  
scaler.fit(train_array)
# 1/10 scale로 train_array 데이터 변환함. 원본 10 -> 1로 변환됨.
train_scaled = scaler.transform(train_array)
 
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]


학습 원본데이터의 경우 1/10로 스케일링 되어서 1은 0.1로, 2는 0.2, 5는 0.5, 10은 1로 변환된다

In [21]:
# 앞에서 생성한 MinMaxScaler에 test_array를 fit() 적용
# 원본 데이터의 최소값이 0, 최대값이 5으로 설정됨 
scaler.fit(test_array)

# 1/5 scale로 test_array 데이터 변환함. 원본 5->1로 변환.  
test_scaled = scaler.transform(test_array)

# train_array 변환 출력
print('원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.2 0.4 0.6 0.8 1. ]


테스트 원본데이터의 경우 1/5로 스케일링 되어서 1은 0.2로, 5는 1로 변환된다  
**--> 학습 데이터와 테스트 데이터의 스케일링이 맞지 않다**

#### 테스트 데이터에 학습데이터로 fit()을 수행한 객체의 transform() 적용

In [23]:
scaler = MinMaxScaler()
scaler.fit(train_array)
train_scaled = scaler.transform(train_array)
print('원본 train_array 데이터:', np.round(train_array.reshape(-1), 2))
print('Scale된 train_array 데이터:', np.round(train_scaled.reshape(-1), 2))

# test_array에 Scale 변환을 할 때는 반드시 fit()을 호출하지 않고 transform() 만으로 변환해야 함. 
test_scaled = scaler.transform(test_array)

print('\n원본 test_array 데이터:', np.round(test_array.reshape(-1), 2))
print('Scale된 test_array 데이터:', np.round(test_scaled.reshape(-1), 2))

원본 train_array 데이터: [ 0  1  2  3  4  5  6  7  8  9 10]
Scale된 train_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]

원본 test_array 데이터: [0 1 2 3 4 5]
Scale된 test_array 데이터: [0.  0.1 0.2 0.3 0.4 0.5]


학습 데이터, 테스트 데이터 모두 1/10 수준으로 스케일링되었다.

**fit_transform() 적용 시에도 테스트 데이터에서는 절대 사용하면 안됨**  
따라서 학습데이터, 테스트 데이터 세트로 분리하기 전에 먼저 **전체 데이터 세트에 fit_transform()으로 스케일링 적용**한 후에 분리